# Creating a Cutout with ERA5

In this example we download ERA5 data on-demand for a cutout we want to create.
(atlite does also work with other datasources, but ERA5 is the easiest one to get started.)

This only works if you have in before

* Installed the Copernicus Climate Data Store `cdsapi` package
* Registered and setup your CDS API key as described [on their website here](https://cds.climate.copernicus.eu/api-how-to)

Import the package first:

In [1]:
import atlite

We implement notifications in `atlite` using loggers from the `logging` library.

We recommend you always launch a logger to get information on what is going on.
For debugging, you can use the more verbose `level=logging.DEBUG`:

In [2]:
import logging

logging.basicConfig(level=logging.INFO)

## Defining the Cutout extent

> This will not yet trigger any major operations.

A cutout is the basis for any of your work and calculations.

The `cutout` is created in the directory and file specified by the relative `path`
If a cutout at the given location already exists, then this command will simply load the cutout again.
If the cutout does not yet exist, it will specify the new to-be-created cutout.
> **Note** `ERA5`, Before the data can be downloaded it has to be processed by CDS servers, this might take a while depending on the ammout of data requested. 
You can check the status of your request [here](https://cds.climate.copernicus.eu/cdsapp#!/yourrequests).

In [3]:
cutout = atlite.Cutout(
    path="western-europe-2011-01.nc",
    module="era5",
    x=slice(-13.6913, 1.7712),
    y=slice(49.9096, 60.8479),
    time="2011-01",
)

INFO:atlite.cutout:Building new cutout western-europe-2011-01.nc


For creating the cutout, you need to specify

* The dataset to create the cutout with (`era5`)
* The time period it covers
* The longitude `x` and latitude `y` it stretches


Here we went with the `ERA5` dataset from ECMWF

```
module="era5"
```

Here we decided to provide the `time` period of the cutout as a string, because it is only a month.
You could have also specify it as a time range

```
slice("2011-01","2011-01")
```

The regional bounds (space the cutout stretches) where specified by the
```
x=slice(-13.6913, 1.7712) # Longitude
y=slice(49.9096, 60.8479) # Latitude
```

and describe a rectangle's edges.
In this case we drew a rectangle containing some parts of the atlantic ocean,
the Republic of Ireland and the UK.

## Preparing the Cutout

If the cutout does not yet exist or has some features which are not yet included, we have to tell atlite to go ahead and do so.

No matter which dataset you use, this is where all the work actually happens.
This can be fast or take some or a lot of time and resources, among others depending on
your computer ressources and (for downloading e.g. ERA5 data) your internet connection.

In [4]:
cutout.prepare()

INFO:atlite.data:Storing temporary files in C:\Users\marta\AppData\Local\Temp\tmphwibk196
INFO:atlite.data:Calculating and writing with module era5:


INFO:atlite.datasets.era5:Requesting data for feature temperature...
INFO:atlite.datasets.era5:Requesting data for feature runoff...
INFO:atlite.datasets.era5:Requesting data for feature height...
INFO:atlite.datasets.era5:Requesting data for feature influx...
INFO:atlite.datasets.era5:Requesting data for feature wind...
2025-02-08 18:38:40,388 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-02-08 18:38:40,389 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using the correct syntax for your API request.
2025-02-08 18:38:40,393 WARNING [2024-06-16T00:00:00] CDS API syntax is chang

<Cutout "western-europe-2011-01">
 x = -13.50 ⟷ 1.75, dx = 0.25
 y = 50.00 ⟷ 60.75, dy = 0.25
 time = 2011-01-01 ⟷ 2011-01-31, dt = H
 module = era5
 prepared_features = ['height', 'wind', 'influx', 'temperature', 'runoff']

The `cutout.prepare()` function takes a list of features which should be prepared. When this is not specified, all available features are build. 

After, the execution all downloaded data is stored at `cutout.path`. Per default it is not loaded into memory, but into [dask](https://dask.org/) arrays. This keeps the memory consumption extremely low.  

The data is accessible in `cutout.data` which is an `xarray.Dataset`. 
Querying the cutout gives us some basic information on which data is contained
in it.

In [5]:
cutout.data

<xarray.Dataset> Size: 130MB
Dimensions:               (x: 62, y: 44, time: 744)
Coordinates:
  * x                     (x) float64 496B -13.5 -13.25 -13.0 ... 1.25 1.5 1.75
  * y                     (y) float64 352B 50.0 50.25 50.5 ... 60.25 60.5 60.75
  * time                  (time) datetime64[ns] 6kB 2011-01-01 ... 2011-01-31...
    lon                   (x) float64 496B dask.array<chunksize=(62,), meta=np.ndarray>
    lat                   (y) float64 352B dask.array<chunksize=(44,), meta=np.ndarray>
Data variables: (12/15)
    height                (y, x) float32 11kB dask.array<chunksize=(44, 62), meta=np.ndarray>
    wnd100m               (time, y, x) float32 8MB dask.array<chunksize=(100, 44, 62), meta=np.ndarray>
    wnd_shear_exp         (time, y, x) float32 8MB dask.array<chunksize=(100, 44, 62), meta=np.ndarray>
    wnd_azimuth           (time, y, x) float32 8MB dask.array<chunksize=(100, 44, 62), meta=np.ndarray>
    roughness             (time, y, x) float32 8MB dask.array<chunksize=(100, 44, 62), meta=np.ndarray>
    influx_toa            (time, y, x) float32 8MB dask.array<chunksize=(100, 44, 62), meta=np.ndarray>
    ...                    ...
    solar_altitude        (time, y, x) float64 16MB dask.array<chunksize=(100, 22, 62), meta=np.ndarray>
    solar_azimuth         (time, y, x) float64 16MB dask.array<chunksize=(100, 22, 62), meta=np.ndarray>
    temperature           (time, y, x) float32 8MB dask.array<chunksize=(100, 44, 62), meta=np.ndarray>
    soil temperature      (time, y, x) float32 8MB dask.array<chunksize=(100, 44, 62), meta=np.ndarray>
    dewpoint temperature  (time, y, x) float32 8MB dask.array<chunksize=(100, 44, 62), meta=np.ndarray>
    runoff                (time, y, x) float32 8MB dask.array<chunksize=(100, 44, 62), meta=np.ndarray>
Attributes:
    module:                  era5
    prepared_features:       ['temperature', 'height', 'runoff', 'influx', 'w...
    chunksize_time:          100
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-01-13T11:28 GRIB to CDM+CF via cfgrib-0.9.1...

We can again breakdown which data array belongs to which feature.

In [6]:
cutout.prepared_features

module  feature    
era5    height                       height
        wind                        wnd100m
        wind                  wnd_shear_exp
        wind                    wnd_azimuth
        wind                      roughness
        influx                   influx_toa
        influx                influx_direct
        influx               influx_diffuse
        influx                       albedo
        influx               solar_altitude
        influx                solar_azimuth
        temperature             temperature
        temperature        soil temperature
        temperature    dewpoint temperature
        runoff                       runoff
dtype: object

If you have matplotlib installed, you can directly use the 
plotting functionality from `xarray` to plot features from
the cutout's data.

> **Warning**
>  This will trigger `xarray` to load all the corresponding data from disk into memory!



Now that your cutout is created and prepared, you can call conversion functions as `cutout.pv` or `cutout.wind`. Note that this requires a bit more information, like what kind of pv panels to use, where do they stand etc. Please have a look at the other examples to get a picture of application cases.

## Reducing Cutout file sizes

Cutouts can become quite large, depending on the spatial and temporal scope they cover.
By default `atlite` uses a trade-off between speed and compression to reduce the file size of cutouts.

Stronger compression can be selected when creating a new cutout by choosing a higher `complevel` (`1` to `9`, default: `4`)
```
cutout.prepare(compression={"zlib": True, "complevel": 9})
```

To change the compression for an existing cutout:
```
cutout = atlite.Cutout("cutout-path.nc")

compression = {"zlib": True, "complevel": 9}
for var in cutout.data.data_vars:
    cutout.data[var].encoding.update(compression)

cutout.to_file()
```
For details and more arguments for `compression`, see the [xarray documentation](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.to_netcdf.html) for details.

Alternatively a cutout can also be compressed by using the `netcdf` utility `nccopy` from the commandline:

```
nccopy -d4 -s <input cutout .nc file> <output cutout .nc file>
```